<a href="https://colab.research.google.com/github/peculab/EduData/blob/main/Example5_RealData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

金門縣中正國小老師提供數據
中正國小六年美班學生使用數位平台（因材網）對數學學習成效之影響

In [44]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [45]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1f8v74oUL3ipebrNfRmuvV2e_S-QMj_HkYEpT_UvH3_I/edit?usp=sharing')

In [46]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,年級,班級,座號,任務進行中,任務完成,任務未完成,影片瀏覽,練習題測驗,AI學習夥伴,使用總時數,平時一,平時二,期中考
0,6,9,1,0,5,1,04:56:12,00:20:28,00:24:11,05:40:51,90,92,85
1,6,9,2,0,0,6,00:01:00,00:00:00,00:00:00,00:01:00,74,81,69
2,6,9,3,0,4,2,06:47:39,00:17:10,00:11:13,07:16:02,85,81,83
3,6,9,4,0,4,2,03:02:23,00:16:28,00:42:52,04:01:43,91,92,96
4,6,9,5,0,6,0,12:35:56,01:13:28,00:36:05,14:25:29,97,96,93


In [47]:
df['平時一'] = df['平時一'].astype(int)
df['平時二'] = df['平時二'].astype(int)
df['期中考'] = df['期中考'].astype(int)

In [48]:
#Data Cleaning - Convert time strings into total seconds for calculations
def time_to_seconds(time_str):
    if isinstance(time_str, int):  # 如果已經是整數，直接返回
        return time_str
    if time_str == "00:00:00":
        return 0
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

In [49]:
for col in ["影片瀏覽", "練習題測驗", "AI學習夥伴", "使用總時數"]:
    df[col] = df[col].apply(time_to_seconds)

In [50]:
#Compute Derived Variables
df["任務完成率"] = df["任務完成"].astype(int) / (df["任務完成"].astype(int) + df["任務未完成"].astype(int))

In [51]:
#Perform Descriptive Statistics
df.describe()

,影片瀏覽,練習題測驗,AI學習夥伴,使用總時數,平時一,平時二,期中考,任務完成率
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,13729.888889,1275.833333,1045.444444,16051.166667,87.722222,87.111111,86.111111,0.416667
std,12247.808312,1189.076223,1015.661374,13401.683232,7.184997,8.316453,10.168515,0.388772
min,0.000000,0.000000,0.000000,0.000000,74.000000,70.000000,62.000000,0.000000
25%,4541.750000,316.000000,16.500000,7038.000000,84.250000,81.750000,80.750000,0.000000
50%,10814.500000,1131.500000,873.000000,12625.000000,89.500000,87.500000,89.500000,0.333333
75%,20238.500000,1451.000000,1513.250000,24209.250000,94.000000,93.500000,93.750000,0.791667
max,45356.000000,4408.000000,3094.000000,51929.000000,97.000000,97.000000,99.000000,1.000000


In [52]:
#Calculate Correlations
correlations = df[["任務完成率", "影片瀏覽", "練習題測驗", "AI學習夥伴", "使用總時數", "平時一", "平時二", "期中考"]].corr()

In [53]:
correlations

,任務完成率,影片瀏覽,練習題測驗,AI學習夥伴,使用總時數,平時一,平時二,期中考
任務完成率,1.000000,0.691971,0.355750,0.579330,0.707862,0.549278,0.582194,0.483595
影片瀏覽,0.691971,1.000000,0.623154,0.336952,0.994727,0.492430,0.381991,0.364605
練習題測驗,0.355750,0.623154,1.000000,0.255448,0.677586,0.445678,0.234103,0.362338
AI學習夥伴,0.579330,0.336952,0.255448,1.000000,0.406392,0.570727,0.628057,0.704794
使用總時數,0.707862,0.994727,0.677586,0.406392,1.000000,0.532828,0.417471,0.418775
平時一,0.549278,0.492430,0.445678,0.570727,0.532828,1.000000,0.777265,0.702522
平時二,0.582194,0.381991,0.234103,0.628057,0.417471,0.777265,1.000000,0.652312
期中考,0.483595,0.364605,0.362338,0.704794,0.418775,0.702522,0.652312,1.000000


In [54]:
# 只選取數字型欄位
numeric_cols = df.select_dtypes(include=["number"]).columns
grouped = df.groupby(pd.cut(df["任務完成率"], bins=[0, 0.5, 0.8, 1.0]))[numeric_cols].mean()
grouped

<ipython-input-54-79260a1c8c37>:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,影片瀏覽,練習題測驗,AI學習夥伴,使用總時數,平時一,平時二,期中考,任務完成率
任務完成率,,,,,,,,
"(0.0, 0.5]",5149.000000,1053.25,1266.500000,7468.750000,89.250000,87.5,84.0,0.250000
"(0.5, 0.8]",15362.666667,962.00,1232.666667,17557.333333,83.666667,83.0,85.0,0.666667
"(0.8, 1.0]",26948.600000,1972.40,1802.600000,30723.600000,94.600000,95.8,93.6,0.900000


1. 影片瀏覽時間分佈直方圖

    圖表內容：該直方圖展示學生觀看學習影片的總時數分佈情況，x軸為影片瀏覽時間（秒），y軸為人數。
    圖表解釋：
        多數學生的影片瀏覽時間集中在某一範圍內，可能顯示部分學生僅完成基礎學習需求。
        瀏覽時間的尾部可能顯示少數學生投入了顯著更多的時間，代表學習動機較高或需要更多時間來掌握內容。
        這樣的分佈可用來區分「低學習投入」、「中學習投入」和「高學習投入」的學生群體，進一步分析這些群體的學習成效差異。

In [55]:
import plotly.express as px

fig = px.histogram(df, x="影片瀏覽", nbins=10, title="影片瀏覽時間分佈")
fig.update_layout(xaxis_title="影片瀏覽時間（秒）", yaxis_title="人數")
fig.show()

2. 不同任務完成率類別的期中考成績分佈（箱型圖）

    圖表內容：該箱型圖展示「任務完成率」分為低、中、高三個類別後，各類別學生的期中考成績分佈。
    圖表解釋：
        低完成率的學生，其期中考成績集中在較低範圍，且分佈範圍較窄，可能反映學習效果不佳。
        中完成率的學生成績有所提升，但分佈較分散，顯示不同學生的學習成效差異大。
        高完成率的學生成績平均值最高，且分佈範圍較窄，表明這些學生不僅學習表現優秀，且較為穩定。
        這表明「任務完成率」與學業成績之間存在正向關聯性。

In [56]:
# 新增任務完成率類別
df["任務完成率類別"] = pd.cut(df["任務完成率"], bins=[0, 0.5, 0.8, 1.0], labels=["低", "中", "高"])

# 繪製箱型圖
fig = px.box(df, x="任務完成率類別", y="期中考", title="不同任務完成率類別的期中考成績分佈")
fig.update_layout(xaxis_title="任務完成率類別", yaxis_title="期中考成績")
fig.show()

3. 影片瀏覽時間與期中考成績的散點圖與迴歸線

    圖表內容：該圖表展示影片瀏覽時間（x軸）與期中考成績（y軸）之間的散點分佈及其線性回歸趨勢。
    圖表解釋：
        從散點分佈中可以觀察到一定的正相關趨勢，表明影片瀏覽時間較長的學生期中考成績較高。
        然而，圖中可能存在一些異常值，例如影片瀏覽時間極長但成績較低的學生，可能需要進一步檢查他們是否有效利用了學習資源。
        回歸線的斜率可用來定量描述影片瀏覽時間對成績的影響程度。
        此結果支持影片瀏覽時間作為學業成績的一項重要預測指標，但同時提示單純增加學習時間可能不一定能有效提升成績。

In [57]:
fig = px.scatter(df, x="影片瀏覽", y="期中考", trendline="ols", title="影片瀏覽時間與期中考成績的相關性")
fig.update_layout(xaxis_title="影片瀏覽時間（秒）", yaxis_title="期中考成績")
fig.show()

圖表解釋：平台使用行為的分群結果
圖表內容

    軸標註解：
        X軸：練習題測驗時間（秒），代表學生完成平台上練習題的總投入時間。
        Y軸：AI學習夥伴互動時間（秒），反映學生與AI學習夥伴互動的總時數。
    顏色區分：不同顏色的點表示分群結果，每個群組代表一種平台使用模式。
    點大小：每個點的大小與學生的期中考成績相關，成績越高，點越大。

圖表解讀

    分群結構：
        群組0（深紫色）：
            學生在練習題測驗和AI學習夥伴互動上的時間均較少。
            該群組的學生可能學習投入度偏低，可能需要更多的引導和激勵。
        群組1（黃色）：
            學生的AI學習夥伴互動時間較長，但練習題測驗時間中等。
            該群組可能偏重於與AI互動，但在練習題實際應用方面有待提高。
        群組2（粉紅色）：
            學生的練習題測驗時間和AI學習夥伴互動時間均較高。
            該群組的學生顯示出高學習投入度，且期中考成績整體較高，可能代表較佳的學習模式。

    期中考成績與學習行為的關聯：
        點的大小顯示期中考成績：
            群組2的點較大，表示投入時間較高的學生，其數學成績表現優異。
            群組0的點較小，暗示投入不足的學生，成績偏低。
        此結果反映「練習題測驗」和「AI學習夥伴互動」時間對數學學業表現可能有正向影響。

    分群的教育意義：
        群組0學生需要教師更多的關注，可能透過設計更有趣的任務提升其參與度。
        群組1學生可能需要更多練習題的實戰訓練，以平衡其學習模式。
        群組2的學習模式值得推廣，例如更頻繁的練習題測驗和AI互動時間，幫助學生獲得更高的學習成效。

建議

    根據分群結果，教師可以針對不同群組設計差異化教學策略。
    平台開發者可以進一步研究不同群組的行為特徵，優化平台功能以支持個性化學習。
    鼓勵學生平衡練習與互動，提升學習效率，進而提升整體學業表現。

這張圖表的分群結果為教育策略提供了具體方向，幫助針對不同學習行為進行有針對性的干預。

In [58]:
from sklearn.cluster import KMeans

# 選擇特徵進行分群
X = df[["練習題測驗", "AI學習夥伴"]]
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
df["群組"] = kmeans.labels_

# 繪製散點圖（群組分佈）
fig = px.scatter(df, x="練習題測驗", y="AI學習夥伴", color="群組", size="期中考",
                    title="平台使用行為的分群結果")
fig.update_layout(scene=dict(
    xaxis_title="影片瀏覽時間（秒）",
    yaxis_title="練習題測驗時間（秒）",
    zaxis_title="AI學習夥伴（秒）"
))
fig.show()

分析說明
一、提案議題

中正國小六年美班學生使用數位平台（因材網）對數學學習成效之影響

二、動機與目的

動機

    在「生生有平板」政策的推動下，科技化教學已成為教育趨勢。本研究針對學生使用因材網平台的行為模式進行分析，以了解數位平台對數學學習成效的影響，為數位教學實踐提供參考。

目的

    描述六年美班學生使用因材網的現況，包括使用時間、任務完成情況等。
    分析學生平台使用行為與數學學業成績之間的關聯性。

三、數據準備與處理

數據來源

    學生平台使用資料：任務完成數、任務未完成數、影片瀏覽時間、練習題測驗時間、AI學習夥伴互動時間、使用總時數。
    學生成果評量：113學年度六年級數學平時考與期中考成績。

資料處理步驟

    資料整合
    使用座號作為識別碼，整合平台使用資料與學生成績資料，形成完整數據集。
    資料清理
        缺失值處理：若數據不完整，考慮以平均值或中位數填補，或移除該筆資料。
        異常值處理：檢查並處理不合理的數據（如過高的瀏覽時間），決定移除或調整。
    變數計算與轉換
        計算「任務完成率」= 任務完成數 / (任務完成數 + 任務未完成數)。
        將連續變數（影片瀏覽時間、練習題測驗時間、使用總時間等）轉換為高、中、低三類，適合分群或非線性模型分析。

四、分析方法與流程

分析方法

    描述性統計分析
    計算平均數、標準差，了解學生平台使用行為及學業成績的分佈特性。
    相關分析
    使用皮爾森相關係數或斯皮爾曼等級相關係數，分析平台使用變數（如影片瀏覽時間、任務完成率）與數學成績的關聯性。
    迴歸分析
        以數學成績（平時考、期中考）為應變數，建立迴歸模型，探討平台使用行為的影響。
        考慮線性迴歸或邏輯迴歸模型，根據數據特性選擇最佳模型。
    分群分析（集群分析）
    根據平台使用行為進行學生分群，分析不同使用模式與學業成效的差異。

分析流程

    數據準備與清理。
    進行描述性統計，檢視數據分佈。
    分析各變數與學業成績的相關性。
    建立迴歸模型，量化平台行為對學業成績的影響。
    進行分群分析，探討學生使用平台的行為模式及其學業表現差異。
    根據分析結果，提出數位教學改進建議。

五、初步結果與預期效益

初步結果

    數據顯示「任務完成率」較高的學生，其平時考與期中考成績平均分數顯著較高。
    影片瀏覽時間與數學成績呈現中等正相關，顯示學生的學習投入度可能影響學業表現。

預期效益

    提供教師數位教學策略的實證依據。
    幫助教師根據學生的使用行為，量身打造個性化學習支持方案。
    提升學生的自主學習能力，促進數學學習動機與成效。
    為平台開發者提供優化建議，使系統更符合學生的實際需求。

資料視覺化示例

    影片瀏覽時間分佈直方圖
    展示學生學習投入的分佈情況。
    任務完成率與期中考成績的箱型圖
    分析不同完成率群體的數學成績分佈。
    影片瀏覽時間與期中考成績的散點圖與迴歸線
    探討學習投入時間與成績的相關性。

這些分析將有助於揭示平台使用行為對學習成效的影響，為數位教學提供重要參考依據。